# Audio MNIST

In [1]:
# Import python packages
from setup import Setup
from utilities import Utilities
import glob
import os
import pandas as pd
from data_processing import DataProcessing
from data_visualization import DataVisualization
from feature_engineering import FeatureEngineering

# Initialize the setup class with the config file path
SU = Setup(cfg_filepath = 'config.yaml')

# Get the paths
source_path = SU.source_path
meta_data_path = SU.source_meta_path
destination_path = SU.destination_path
plot_path = SU.plot_path
meta_data = SU.meta_data

# Initialize the Utilities class
UT = Utilities()

# Define the target sample rate
target_sr = 8000

# Initialize the DataProcessing class with the destination path, and target sample rate
DP = DataProcessing(destination_path, target_sr)

# Initialize the DataVisualization class with the target sample rate
DV = DataVisualization()

# Create empty dataframe with column names
columns=['mean', 'std', 'avg', 'median', 'min', 'max', 'skewness', 'kurtosis', 'gender', 'digit']
df = UT.create_dataframe(columns)

# Specify total number of folders in source path
all_folders = len(next(os.walk(source_path))[1])+1

# Loop over audio recordings in the source path
for i in range(1, all_folders):
    # Show progress
    UT.loop_progress(i, all_folders-1)

    # Assign source temp
    src_temp = os.path.join(source_path, f"{i:02d}")
    filepath_filename = sorted(glob.glob(os.path.join(src_temp, "*.wav")))

    # Loop over files in directory
    for file in filepath_filename:
        # Split file string
        dig, vp, rep = file.rstrip(".wav").split("/")[-1].split("_")

        # Read audio data
        fs, data = UT.read_audio(file)

        # Plot audio signal
        plot_name = f"{dig[-1]}_{vp}_{rep}.png"
        #DV.plot_signal(fs, data, plot_path, plot_name)

        # Play audio signal
        #DV.play_audio(file)

        # Resample audio data
        data = DP.resample_data(fs, data, target_sr)

        # Zero padding audio data
        data = DP.zero_pad(data, target_sr)

        # FFT audio data
        data = DP.fft_data(data)

        # Feature creation
        features = DP.feature_creation(data)

        # Normalize features
        n_features = DP.normalize_features(features)

        # Add gender and digit label
        features = DP.add_gender(n_features, meta_data[vp]["gender"])
        features = DP.add_digit(n_features, dig[-1])

        # Append new dict values to the DataFrame
        df = df.append(features, ignore_index=True)

# Save data to CSV
csv_name = "audio_data.csv"
UT.save_df_to_csv(df, destination_path, csv_name)

Progress: 1.67%
Progress: 3.33%
Progress: 5.00%
Progress: 6.67%
Progress: 8.33%
Progress: 10.00%
Progress: 11.67%
Progress: 13.33%
Progress: 15.00%
Progress: 16.67%
Progress: 18.33%
Progress: 20.00%
Progress: 21.67%
Progress: 23.33%
Progress: 25.00%
Progress: 26.67%
Progress: 28.33%
Progress: 30.00%
Progress: 31.67%
Progress: 33.33%
Progress: 35.00%
Progress: 36.67%
Progress: 38.33%
Progress: 40.00%
Progress: 41.67%
Progress: 43.33%
Progress: 45.00%
Progress: 46.67%
Progress: 48.33%
Progress: 50.00%
Progress: 51.67%
Progress: 53.33%
Progress: 55.00%
Progress: 56.67%
Progress: 58.33%
Progress: 60.00%
Progress: 61.67%
Progress: 63.33%
Progress: 65.00%
Progress: 66.67%
Progress: 68.33%
Progress: 70.00%
Progress: 71.67%
Progress: 73.33%
Progress: 75.00%
Progress: 76.67%
Progress: 78.33%
Progress: 80.00%
Progress: 81.67%
Progress: 83.33%
Progress: 85.00%
Progress: 86.67%
Progress: 88.33%
Progress: 90.00%
Progress: 91.67%
Progress: 93.33%
Progress: 95.00%
Progress: 96.67%
Progress: 98.33%
Pr

In [2]:
# Load CSV file into dataframe
my_df = UT.csv_to_dataframe(destination_path, csv_name)

# Initialize the DataVisualization class with the target sample rate
FE = FeatureEngineering()

# Leave target columns out
columns_to_leave_out = ["gender", "digit"]

# Remove constant columns
my_df = FE.remove_constant_columns(my_df, columns_to_leave_out)

# Calculate correlation matrix
corr_matrix = FE.pearson_correlation(my_df, columns_to_leave_out)

# Assign correlation threshold
threshold = 0.95

# Removed correlated columns
final_df = FE.remove_correlated_columns(my_df, threshold, columns_to_leave_out)

# Save data to CSV
csv_name = "final_data.csv"
UT.save_df_to_csv(final_df, destination_path, csv_name)